In [1]:
using SX, Reachability
using Plots

Open and parse the SX model:

In [3]:
file = "platoonComp_flat_revised.xml"

HDict = readsxmodel(file, raw_dict=true);
H = readsxmodel(file, ST=ConstrainedLinearControlContinuousSystem);

[warn | Reachability]: field note in location 1 is ignored
[warn | Reachability]: field note in location 2 is ignored
[warn | Reachability]: field label in transition 1 → 2 is ignored
[warn | Reachability]: field labelposition in transition 1 → 2 is ignored
[warn | Reachability]: field label in transition 2 → 1 is ignored
[warn | Reachability]: field labelposition in transition 2 → 1 is ignored
[warn | Reachability]: field middlepoint in transition 2 → 1 is ignored
[warn | Reachability]: field note in location 1 is ignored
[warn | Reachability]: field note in location 2 is ignored
[warn | Reachability]: field label in transition 1 → 2 is ignored
[warn | Reachability]: field labelposition in transition 1 → 2 is ignored
[warn | Reachability]: field label in transition 2 → 1 is ignored
[warn | Reachability]: field labelposition in transition 2 → 1 is ignored
[warn | Reachability]: field middlepoint in transition 2 → 1 is ignored


In [6]:
HDict["variables"]

DataStructures.OrderedDict{Symbol,Dict{String,Any}} with 11 entries:
  :t   => Dict{String,Any}(Pair{String,Any}("local", false),Pair{String,Any}("c…
  :xe1 => Dict{String,Any}(Pair{String,Any}("local", false),Pair{String,Any}("c…
  :v1  => Dict{String,Any}(Pair{String,Any}("local", false),Pair{String,Any}("c…
  :a1  => Dict{String,Any}(Pair{String,Any}("local", false),Pair{String,Any}("c…
  :xe2 => Dict{String,Any}(Pair{String,Any}("local", false),Pair{String,Any}("c…
  :v2  => Dict{String,Any}(Pair{String,Any}("local", false),Pair{String,Any}("c…
  :a2  => Dict{String,Any}(Pair{String,Any}("local", false),Pair{String,Any}("c…
  :xe3 => Dict{String,Any}(Pair{String,Any}("local", false),Pair{String,Any}("c…
  :v3  => Dict{String,Any}(Pair{String,Any}("local", false),Pair{String,Any}("c…
  :a3  => Dict{String,Any}(Pair{String,Any}("local", false),Pair{String,Any}("c…
  :aL  => Dict{String,Any}(Pair{String,Any}("local", false),Pair{String,Any}("c…

In [5]:
H.modes[1].A

10×10 Array{Float64,2}:
 0.0  0.0       0.0       0.0        …   0.0       0.0        0.0      
 0.0  0.0       1.0       0.0            0.0       0.0        0.0      
 0.0  0.0       0.0      -1.0            0.0       0.0        0.0      
 0.0  1.60499   4.86799  -3.57539       -0.194199  0.362599  -0.0945999
 0.0  0.0       0.0       0.0            0.0       0.0        0.0      
 0.0  0.0       0.0       1.0        …   0.0       0.0        0.0      
 0.0  0.871799  3.81399  -0.0753999     -0.594999  0.129399  -0.0795999
 0.0  0.0       0.0       0.0            0.0       1.0        0.0      
 0.0  0.0       0.0       0.0            0.0       0.0       -1.0      
 0.0  0.713199  3.57299  -0.0963999      1.27259   3.07199   -3.13559  

In [ ]:
H.resetmaps[1].A

In [ ]:
H.resetmaps[1].B

In [ ]:
H.modes[1].X

In [ ]:
H.modes[1].B

In [ ]:
H.modes[1].A[4, :]

In [ ]:
H.ext[:variables]

In [ ]:
A = H.modes[1].A[2:end, 2:end]

# S = ConstrainedLinearControlContinuousSystem(H.modes[1].A[1:6, 1:6], eye(6), nothing, ConstantInput(H.modes[1].B[1:6] * H.modes[1].U[1]))

In [ ]:
HDict = readsxmodel(file, raw_dict=true)

In [ ]:
N = Float64
variables = HDict["variables"]
invariants = HDict["invariants"]
flows = HDict["flows"]
assignments = HDict["assignments"]
guards = HDict["guards"]
state_variables = Vector{Basic}()
input_variables = Vector{Basic}()

for vi in keys(variables)
   if variables[vi]["controlled"]
       push!(state_variables, convert(Basic, vi))
   else
       push!(input_variables, convert(Basic, vi))
   end
end
m = length(input_variables)

In [ ]:
invariants = HDict["invariants"]

In [ ]:
invi = invariants[1][2]

In [ ]:
invi = :(-aL <= 9)

In [ ]:
expr1.args

In [ ]:
LHS = convert(Basic, invi.args[2])
RHS = convert(Basic, invi.args[3])
LHS, RHS

In [ ]:
b = convert(N, RHS)

In [ ]:
U = Vector{LazySet{N}}() # FIXME : use an intersection array?
# should be a vector of vectors (one for each location)

# vector of state constraints
X = Vector{LazySet{N}}() # FIXME : use an intersection array?

# find if this is a state constraint or an input constraint
if all(si in state_variables for si in free_symbols(LHS))
    a = diff.(LHS, state_variables)
    a = convert(Vector{N}, a)
    push!(X, HalfSpace(a, b))
elseif all(si in input_variables for si in free_symbols(LHS))
    a = diff.(LHS, input_variables)
    a = convert(Vector{N}, a)
    push!(U, HalfSpace(a, b))
else
    error("the invariant $invi is not in the specified format")
end

In [ ]:
H = readsxmodel(file, ST=ConstrainedLinearControlContinuousSystem)

Check that the state variables are those betweeen 1 and 6:

In [ ]:
keys(H.ext[:variables])

Build the reachability problem, as an initial value problem with uncertain initial states and inputs:

In [ ]:
S = ConstrainedLinearControlContinuousSystem(H.modes[1].A[1:6, 1:6], eye(6), nothing, ConstantInput(H.modes[1].B[1:6] * H.modes[1].U[1]))

initially = " x1 >= 0.0060145 & x1 <= 0.0115005 & x2 >= -0.0009624 & x2 <= 0.0025689 & x3 >= 0.0177578 & x3 <= 0.0257479 & x4 >= -0.0047553 & x4 <= -0.0001524 & x5 >= 0.0177717 & x5 <= 0.0371327 & x6 >= -0.0122932 & x6 <= -0.0046144 & y1 >= -0.0008518 & y1 <= 0.0001322 & u1 >= 0.8000000 & u1 <= 1.0000000 & t==0 & stoptime == 20.00"
f = i -> SX.parse_sxmath(initially)[i].args[3]
X0 = Hyperrectangle(low=[f(i) for i in 1:2:12], high=[f(i) for i in 2:2:12])

problem = InitialValueProblem(S, X0);

Solve the problem in verbose mode:

In [ ]:
sol = solve(problem, :T=>5.0, :δ=>0.01, :vars=>[6], :verbosity=>1);

In [ ]:
plot(sol, alpha=0.2)

Now compute the output $y_1$:

In [ ]:
vars = collect(keys(H.ext[:variables]))[1:6]
expr = SX.parse_sxmath("y1 == 0.064531*x2 - 0.0061322*x1 - 0.00069603*x3 - 0.062226*x4 - 0.0035048*x5 + 0.047228*x6")
Y1 = convert(Vector{Float64}, diff.(convert(Basic, expr[1].args[2]), vars))
Y1 = sparse(reshape(Y1, (1, 6)))

# (first input in plot_vars measn time; second input is ignored if projection matrix is passed)
sol = solve(problem, :T=>5.0, :δ=>0.01, :plot_vars=>[0, 1], :vars=>1:6, :verbosity=>1, :projection_matrix=>Y1);

In [ ]:
plot(sol, alpha=0.2, xlabel="t", ylabel="y1")

In [ ]:
function reach_hybrid(As, Ts, init, δ, μ, T, max_order, must_semantics)
    # initialize queue with initial mode and states at time t=0
    queue = [(init[1], init[2], 0.)]

    res = Tuple{LazySet, Int}[]
    while !isempty(queue)
        init, loc, t = pop!(queue)
        println("currently in location $loc at time $t")
        R = reach_continuous(As[loc], init, δ, μ, T-t, max_order)
        found_transition = false
        for i in 1:length(R)-1
            S = R[i]
            push!(res, (S, loc))
            for (guard, tgt_loc) in Ts[loc]
                if !is_intersection_empty(S, guard)
                    new_t = t + δ * i
                    push!(queue, (S, tgt_loc, new_t))
                    found_transition = true
                    println("transition $loc -> $tgt_loc at time $new_t")
                end
            end
            if must_semantics && found_transition
                break
            end
        end
        if !must_semantics || !found_transition && length(R) > 0
            push!(res, (R[end], loc))
        end
    end
    return res
end